In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [3]:
spark = SparkSession \
    .builder \
    .appName("aggtaxiData") \
    .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.3.2/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/fdr/.ivy2/cache
The jars for the packages stored in: /Users/fdr/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7712ea61-a6ff-4420-88e1-641bb324089a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apa

23/03/20 19:17:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
fhv_spark = T.StructType([
    T.StructField("dispatching_base_num", T.StringType(), True),
    T.StructField("pickup_datetime", T.StringType(), True),
    T.StructField("dropOff_datetime", T.StringType(), True),
    T.StructField("PULocationID", T.IntegerType(), True),
    T.StructField("DOLocationID", T.IntegerType(), True),
    T.StructField("SR_Flag", T.IntegerType(), True),
    T.StructField("Affiliated_base_number", T.StringType(), True),
])

In [5]:
green_spark = T.StructType([
    T.StructField("VendorID", T.IntegerType(), True),
    T.StructField("lpep_pickup_datetime", T.StringType(), True),
    T.StructField("lpep_dropoff_datetime", T.StringType(), True),
    T.StructField("store_and_fwd_flag", T.StringType(), True),
    T.StructField("RatecodeID", T.IntegerType(), True),
    T.StructField("PULocationID", T.IntegerType(), True),
    T.StructField("DOLocationID", T.IntegerType(), True),
    T.StructField("passenger_count", T.IntegerType(), True),
    T.StructField("trip_distance", T.FloatType(), True),
    T.StructField("fare_amount", T.FloatType(), True),
    T.StructField("extra", T.FloatType(), True),
    T.StructField("mta_tax", T.FloatType(), True),
    T.StructField("tip_amount", T.FloatType(), True),
    T.StructField("tolls_amount", T.FloatType(), True),
    T.StructField("ehail_fee", T.IntegerType(), True),
    T.StructField("improvement_surcharge", T.FloatType(), True),
    T.StructField("payment_type", T.FloatType(), True),
    T.StructField("trip_type", T.FloatType(), True),
    T.StructField("congestion_surcharge", T.IntegerType(), True)
])

In [6]:
df_fhv_raw = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "fhv_rides_head_json") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint") \
    .option("failOnDataLoss", "false") \
    .load()

In [7]:
df_fhv_raw.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
parsed_fhv = df_fhv_raw \
    .select(F.from_json(F.col("value").cast("string"), fhv_spark).alias("fhv")) \
    .select("fhv.PULocationID")


In [9]:
# aggregated_fhv = parsed_fhv \
#     .groupBy("PULocationID") \
#     .agg({"PULocationID": "count"}) \
#     .withColumnRenamed("count(PULocationID)", "count_PULocs") \
#     .select("PULocationID", "count_PULocs")

In [10]:
# fhv_query = parsed_fhv.writeStream.format("console") \
#             .option("truncate", "false") \
#             .outputMode("append") \
#             .option("checkpointLocation", "checkpoint") \
#             .start() \
#             .awaitTermination()

In [11]:
df_green_raw = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green_rides_head_json") \
    .option("startingOffsets", "earliest") \
    .option("checkpointLocation", "checkpoint_green") \
    .option("failOnDataLoss", "false") \
    .load()

In [12]:
parsed_green = df_green_raw \
    .select(F.from_json(F.col("value").cast("string"), green_spark).alias("green")) \
    .select("green.PULocationID")

In [13]:
# aggregated_green = parsed_green \
#     .groupBy("PULocationID") \
#     .agg({"PULocationID": "count"}) \
#     .withColumnRenamed("count(PULocationID)", "count_PULocs") \
#     .select("PULocationID", "count_PULocs")

In [14]:
# green_query = aggregated_green.writeStream.format("console") \
#             .option("truncate", "false") \
#             .outputMode("complete") \
#             .option("checkpointLocation", "checkpoint_green") \
#             .start() 

In [15]:
# aggregated_fhv = parsed_fhv \
#     .groupBy("PULocationID") \
#     .agg({"PULocationID": "count"}) \
#     .withColumnRenamed("count(PULocationID)", "count_PULocs") \
#     .select("PULocationID", "count_PULocs")

In [16]:
# parsed_green = parsed_green.select("PULocationID")

In [17]:
# aggregated_green = parsed_green \
#     .groupBy("PULocationID") \
#     .agg({"PULocationID": "count"}) \
#     .withColumnRenamed("count(PULocationID)", "count_PULocs") \
#     .select("PULocationID", "count_PULocs")

In [18]:
parsed_df = parsed_fhv.union(parsed_green)


In [19]:
parsed_df.printSchema()

root
 |-- PULocationID: integer (nullable = true)



In [20]:
aggregated_df = parsed_df \
    .groupBy("PULocationID") \
    .agg({"PULocationID": "count"}) \
    .withColumnRenamed("count(PULocationID)", "count_PULocs") \
    .select("PULocationID", "count_PULocs")

In [21]:
aggregated_df.printSchema()

root
 |-- PULocationID: integer (nullable = true)
 |-- count_PULocs: long (nullable = false)



In [22]:
# output_query = aggregated_df \
#     .writeStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "localhost:9092") \
#     .option("topic", "output_topic") \
#     .option("checkpointLocation", "/tmp/checkpoints") \
#     .start()

In [23]:
query = aggregated_df.writeStream.format("console") \
            .option("truncate", "false") \
            .outputMode("complete") \
            .option("checkpointLocation", "checkpoint_df") \
            .start() 

23/03/20 19:17:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [24]:
query.awaitTermination()

23/03/20 19:19:01 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:01 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:01 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:01 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:01 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapsh

23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapsh

23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:02 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapsh

23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapsh

23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapshot file and delta files if needed...Note that this is normal for the first batch of starting query.
23/03/20 19:19:03 WARN HDFSBackedStateStoreProvider: The state for version 4 doesn't exist in loadedMaps. Reading snapsh

...

-------------------------------------------
Batch: 6
-------------------------------------------
+------------+------------+
|PULocationID|count_PULocs|
+------------+------------+
|85          |4           |
|255         |2           |
|76          |2           |
|223         |2           |
|null        |0           |
|189         |10          |
|49          |12          |
|129         |2           |
|97          |10          |
|264         |10          |
|82          |10          |
|25          |2           |
|256         |2           |
|71          |2           |
+------------+------------+



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/3.3.2/libexec/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/Cellar/apache-spark/3.3.2/libexec/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/Cellar/python@3.10/3.10.6_2/Frameworks/Python.framework/Versions/3.10/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

-------------------------------------------
Batch: 7
-------------------------------------------
+------------+------------+
|PULocationID|count_PULocs|
+------------+------------+
|85          |6           |
|255         |3           |
|76          |3           |
|223         |3           |
|null        |0           |
|189         |11          |
|49          |14          |
|129         |3           |
|97          |11          |
|264         |11          |
|82          |11          |
|25          |3           |
|256         |3           |
|71          |3           |
+------------+------------+



-------------------------------------------
Batch: 8
-------------------------------------------
+------------+------------+
|PULocationID|count_PULocs|
+------------+------------+
|85          |8           |
|255         |4           |
|76          |4           |
|223         |4           |
|null        |0           |
|189         |12          |
|49          |16          |
|129         |4           |
|97          |12          |
|264         |12          |
|82          |12          |
|25          |4           |
|256         |4           |
|71          |4           |
+------------+------------+

